# 08MIAR - Aprendizaje Por Refuerzo: Proyecto de programación "*Entrenamiento Agente Para Space Invaders*"



---


##### Autores:    Daniel Romero Martinez / Roberto Vazquez Calvo / Pere Marc Monserrat Calbo
##### Fecha:      Enero 2026.
##### Titulación: Master Universitario en Inteligencia Artificial - VIU
##### Profesor:   D. Jose Manuel Camacho

---



---
# 0. Enunciado del Problema

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga **alcanzar más de 20 puntos con reward clipping durante más de 100 episodios consecutivos**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
# 1. Instalación y preparacion del entorno
Todo el codigo que prepara el entorno en base a si estamos en Colab o en Local se ha llevado a una funcion ejecutable para mantener el notebook mas limpio.

In [1]:
# Llamamos a funcion que prepara el entorno.
!python entorno_spaceinvadersv0.py


Estamos en entorno Local: /home/rober/anaconda3/envs/GymClasic/bin/python
Directorio actual: /home/rober/proyectos_python/08MIAR_AprendizajePorRefuerzo/Portfolio/Proyecto_Final
Archivos en el directorio:
['Solucion1', 'entorno_spaceinvadersv0.py', 'Solucion3', 'Anexo_MEMORIA.md', '__pycache__', 'img', 'env_functions.py', '08MIAR_Proyecto_Programacion_G21_Final.ipynb', '.ipynb_checkpoints']

La version de Python instalada en este entorno Local es: 3.8

Comprobando Paquetes Instalados....

✔ numpy ya está instalado.
✔ gym ya está instalado.
✔ atari_py ya está instalado.
✔ pyglet ya está instalado.
✔ h5py ya está instalado.
✔ Pillow ya está instalado.
✔ keras-rl2 ya está instalado.
✔ Keras ya está instalado.
✔ tensorflow ya está instalado.
✔ opencv-python-headless ya está instalado.
✔ matplotlib ya está instalado.

El entorno esta listo.


---
# 2. Desarrollo y preguntas

## 2.1 Declaraciones Generales

In [3]:
# Llamada a funciones generales
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input, Layer, Permute

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory, Memory

from IPython.display import display, Markdown

# Llamada a funciones propias
from env_functions import plot_rewards, agent_eval, ShowLastTraining, merge_json_logs
from env_functions import set_seeds, make_env, AtariProcessor, TrainAgent, TestAgent
import env_functions as ef

# Anulamos UserWarnings de TensorFlow
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="tensorflow")


## 2.3 Solución nº2 - Double DQN
En esta solucion implementamos una arquitectura doble DQN...
<p align="center">
  <img src="img/Double_DQN_img.png" width="600">
</p>

La red Double DQN ....

#### 2.3.1 Configuración base

In [17]:
# Definicion de parametros fijos y carga de los entornos de training & test.
# Parametros fijos
seed = 42
Memory_Size = 500000
N_Solucion = 2
Reward_min = 20
Episodios_min = 100
Training_Steps = 1500000

# Definimos la semilla para la reproducibilidad de la ejecución
set_seeds(seed)

# Cargamos entornos de simulacion
env_tr = make_env(seed, training=True)
env_te = make_env(seed, training=False)

# Extraemos informacion sobre las acciones y el espacio de observacion del entorno
nb_actions = env_tr.action_space.n
height, width, channels = env_tr.observation_space.shape

print(f'Numero de acciones del juego: {nb_actions}')
print(f'Tipo de acciones disponibles en el juego: {env_tr.unwrapped.get_action_meanings()}')
print(f'Dimensiones del Frame del juego: [{height},{width},{channels}]')

Numero de acciones del juego: 6
Tipo de acciones disponibles en el juego: ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']
Dimensiones del Frame del juego: [210,160,3]


#### 2.3.2 Implementación de la red neuronal

In [18]:
# Definimos la arquitectura de red neuronal en una funcion, asi podemos llamarla directamente y pasarle los parametros que vamos a evaluar.
def BuildDcnn(n_classes=6):
    model = Sequential(name= 'Double DQN_CNN')
    # Entrada
    model.add(Permute((2, 3, 1), input_shape=(ef.WINDOW_LENGTH,) + ef.INPUT_SHAPE))

    # Base Model - Bloque convolucional
    model.add(Conv2D(16, (8, 8), strides=(4, 4), padding='valid', activation="relu", name="conv1"))
    model.add(Conv2D(32, (4, 4), strides=(2, 2), padding='valid', activation="relu", name="conv2"))
    model.add(Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation="relu", name="conv3"))
    
    # Top Model - Capa FC.
    model.add(Flatten(name="flatten"))
    model.add(Dense(128, activation="relu", name="fc_shared"))
    
    # Capa final: Q(s,a) 
    model.add(Dense(n_classes, activation="linear", name="Q_values"))
    
    return model

# mostramos datos entrada
print("Tamaño de la imagen de entrada:", ef.IM_SHAPE)
print("Numero de clases/acciones:", nb_actions)

# Llamamos a la funcion de construccion de la red del modelo. Esta red neuronal: recibe el estado s, produce vector con los valores Q(s,a), en este caso
# tendremos un vector con 6 componentes (6 acciones). La red evalua cual es la mejor Q(s,a).
model = BuildDcnn(nb_actions)

# Mostramos la arquitectura de la red declarada
model.summary()

Tamaño de la imagen de entrada: (84, 84, 4)
Numero de clases/acciones: 6
Model: "Double DQN_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 20, 20, 16)        4112      
_________________________________________________________________
conv2 (Conv2D)               (None, 9, 9, 32)          8224      
_________________________________________________________________
conv3 (Conv2D)               (None, 7, 7, 32)          9248      
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
fc_shared (Dense)            (None, 128)               200832    
_____________________________________________

#### 2.3.3 Implementación de la solución DQN

In [20]:
# Definimos nuestro agente: Buffer memoria, policy y tipo agente.
def BuildAgent(model, memory_size, tr_steps, n_actions, lr=0.0005):
    
    # Preparación de la memoria (usamos memory replay en DQN).
    ReplayBuffer = SequentialMemory(limit=memory_size, window_length=ef.WINDOW_LENGTH)

    # Llamamos a clase para pre-procesado de las observaciones. 
    processor = AtariProcessor()

    # Definimos la policy, estrategia para seleccionar las acciones. Decide que accion tomar.
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(),     # Estrategia E-greedy.
        attr='eps',
        value_max=1.0,          # Explora totalmente al inicio
        value_min=0.05,         # Explora poco al final del entrenamiento
        value_test=0.05,        # Exploración mínima en test
        nb_steps=tr_steps       # Duración del decaimiento
    )
    
    # Definición del agente DQN. Aqui se junta todo, es el cerebro que coordina todo.
    # Llama a la ANN para estimar la Q, aplica la policy para elegir la acción, interactua con el entorno y usa el replay bufer para guardar las experiencias,
    # entrena la red mediante el target network para estabilizar el aprendizaje y la actualiza con la función de perdidas.
    dqn = DQNAgent(
        model= model,                    # La red CNN que estima la distribución Q
        nb_actions= n_actions,           # Número de acciones posibles en el entorno
        memory= ReplayBuffer,            # Replay buffer para almacenar transiciones
        policy= policy,                  # Política/Estrategia
        processor= processor,            # Preprocesado de observaciones
        nb_steps_warmup= 50000,          # Pasos iniciales solo explorando (sin entrenar)
        target_model_update= 10000,      # Frecuencia de actualización del target network
        train_interval= 4,               # Entrenar cada X pasos de interacción
        gamma= 0.99                      # Factor de descuento para recompensas futuras
    )

    # Compilación del agente
    dqn.compile(Adam(learning_rate=lr, clipnorm=10.0), metrics=['mae'])
    
    # Mostrar confirmacion
    print("Agente creado con exito.")
    
    return dqn

# Construimos el agente.
agent = BuildAgent(model, Memory_Size, Training_Steps, nb_actions, 0.00025)


Agente creado con exito.


#### 2.3.4 Entrenamiento del Agente DQN
Configuramos y ejecutamos el entrenamiento del agente. Se realiza un entrenamiento en 3 etapas, con diferente numero de steps e hiperparametros del agente en cada una de ellas. Al final de este a apartado se analizan los resultados.

##### 2.3.4.1 Etapa 1: Entrenamiento desde 0 con 1.5M steps

In [ ]:
# Entrenamos agente.
n_tr = 1
TrainAgent(agent, env_tr, Training_Steps, N_Solucion, n_train=n_tr)


In [ ]:
# Test de performance del agente
n_test= 100
TestAgent(agent, env_te, N_Solucion, n_tr, n_test, Episodios_min, Reward_min, False)


##### 2.3.4.2 Etapa 2: Fine Tuning de 750k steps

In [ ]:
# Fijamos steps.
n_tr = 2
Training_Steps = 750000
decay_steps = int(Training_Steps*0.75)

# Reducimos learning rate para fine‑tuning
LearningRate = 0.000015
K.set_value(agent.model.optimizer.learning_rate, LearningRate)

# Actualizamos estrategia agente
# Continuamos con el epsilon final de la etapa anterior y bajamos a 0.01 con el 75% de los steps.
# El 25% final el epsilon sera muy bajo para entrenar practicamente determinista.

agent.policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr='eps',
    value_max=0.05,      # Continua donde lo habia dejado
    value_min=0.01,      # termina casi determinista (1% aleatorio)
    value_test=0.0,      # test sin azar
    nb_steps=decay_steps
)

# Aumentamos el numero de pasos de actualizacion de la red para darle mas estabilidad.
agent.target_model_update= 20000

# Entrenamos agente.
TrainAgent(agent, env_tr, Training_Steps, N_Solucion, n_train=n_tr)


In [ ]:
# Test de performance del agente
n_test= 100
TestAgent(agent, env_te, N_Solucion, n_tr, n_test, Episodios_min, Reward_min, False)


##### 2.3.4.3 Etapa 3: Entrenamiento final de 500k steps adicionales

In [ ]:
# Fijamos steps.
n_tr = 3
Training_Steps = 500000
decay_steps = int(Training_Steps*0.80)

# Reducimos learning rate para fine‑tuning
LearningRate = 0.00001
K.set_value(agent.model.optimizer.learning_rate, LearningRate)

# Actualizamos estrategia agente
# Subimos el epsilon final de la etapa anterior para permitir exploracion de nuevo (10%) y bajamos a 0.005 con el 80% de los steps.
# El 20% final el epsilon sera muy bajo para entrenar practicamente determinista.

agent.policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr='eps',
    value_max=0.10,      # Explora de nuevo un 10% nuevo
    value_min=0.005,     # termina casi determinista (0.5% aleatorio)
    value_test=0.0,      # test sin azar
    nb_steps=decay_steps
)

# Aumentamos el numero de pasos de actualizacion de la red para darle mas estabilidad.
agent.target_model_update= 20000

# Entrenamos agente.
TrainAgent(agent, env_tr, Training_Steps, N_Solucion, n_train=n_tr)


In [ ]:
# Test de performance del agente
n_test= 100
TestAgent(agent, env_te, N_Solucion, n_tr, n_test, Episodios_min, Reward_min, False)

#### 2.3.5 Justificación de los parámetros seleccionados y de los resultados obtenidos

---

In [ ]:
# Graficamos evolución de las 3 fases del entrenamiento
json_paths = [
    "Solucion2/dqn2_SpaceInvaders-v0_log_1.json",
    "Solucion2/dqn2_SpaceInvaders-v0_log_2.json",
    "Solucion2/dqn2_SpaceInvaders-v0_log_3.json",
]

merge_json_logs(json_paths, "Solucion2/dqn2_SpaceInvaders-v0_log_FULL.json")

plot_rewards("Solucion2/dqn2_SpaceInvaders-v0_log_FULL.json", 0.01)

In [ ]:
# Test de performance del agente durante 300 episodios
n_test= 300
TestAgent(agent, env_te, N_Solucion, n_tr, n_test, Episodios_min, Reward_min, False)


Se ha realizado un entrenamiento por etapas:
- Etapa1: Inicialmente se optó por entrenar el modelo con 1.5M de steps. A la vista de los resultados mostrados en la curva de entrenamiento, se identificó que el modelo mantenia una pendiente ascendente en el reward medio por lo que todavia tenia margen de mejora. Se ha utilizado un learning rate de 0.00025 y una estrategia E-greedy con decaimiento progresivo desde 1 a 0.05. La red target se actualiza cada 10k steps.
- Etapa2: Se optó por una fase de ajuste fino de 750k steps adicionales para maximizar la política de explotación. Se ha utilizado un learning rate de 0.000015 y una estrategia E-greedy con decaimiento progresivo desde 0.05 a 0.01 para el 75% de los steps dejando el 25% final con epsilon fijo para hacer entrenamiento mas determinista. La red target se actualiza cada 20k steps para dar más estabilidad al aprendizaje y ayudar a que el error de los valores Q no oscile tanto.
- Etapa3: En vista a los resultados de la etapa anterior, se decide hacer una ultima etapa adicional de 500k steps. Se ha utilizado un learning rate de 0.00001 y una estrategia E-greedy con decaimiento progresivo desde 0.1 a 0.005 para el 80% de los steps dejando el 20% final con epsilon fijo para hacer entrenamiento mas determinista. La red target se actualiza cada 20k steps igual que en la etapa anterior.

Resultados:
- Tras 1.5M steps de entrenamiento, el test del agente ha dado un resultado de 91/100 episodios por encima de 20 puntos y una media de 27.6 puntos, lo cual no está nada mal. El aprendizaje del agente ha ido mejorando progresivamente a lo largo de los 1.5M steps.
- Tras hacer un fine tunning de 750K steps adicionales el test del agente ha dado un resultado de 95/100 episodios por encima de 20 puntos y la media ha sido de 33.2 puntos. El agente ha mejorado claramente en esta 2ª etapa.
- Finalmente tras 500k steps adicionales el test del agente ha dado un resultado de 100/100 episodios por encima de 20 puntos y la media ha sido de 36.3 puntos. El agente ha mejorado claramente en esta 3ª etapa final hasta superar el reto.
- Ejecutamos un test adicional de 300 episodios para ver comportamiento mas amplio. El resultado es de 300/300 episodios por encima de 20 puntos y la media ha sido de 34.17 puntos.
- Analizando la curva completa de entrenamiento de 2.75M de steps, vemos que el agente ha tenido un progreso claro a través de las 3 etapas desarrolladas. 
